<a href="https://colab.research.google.com/github/luuloi/GWAS_Introduction_2023/blob/main/04_GWAS_Case_Control_Association_Testing_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Task 1: Data Inspection and Basic Descriptive Statistics**

1. After loading your data:
    - Check the last few rows of the `LHON` dataframe.
    - Check the unique values in the `GENO` and `PHENO` columns.
    - Calculate the mean, median, and mode for any numerical column present in the `LHON` dataframe.
    
### **Task 2: Data Visualization**

1. Create a pie chart for the `PHENO` column showcasing the distribution of cases and controls.
2. Plot the distribution (histogram) of any numeric columns present in the dataframe.
3. Create a scatter plot between two numeric columns (if available) and use color to differentiate between the two `PHENO` groups.

### **Task 3: Statistical Analysis**

1. Choose any two categorical columns from the `LHON` dataframe (other than `GENO` and `PHENO`) and create a contingency table.
2. Perform the Chi-square test on this new table and interpret the results.

### **Task 4: Logistic Regression and Interpretation**

1. If you have more categorical predictors, encode them similarly to `genoadd` (using numerical encoding) and add them to your logistic regression model.
2. Interpret the significance of each predictor using the summary results.
3. Predict the probabilities for the training set using your model and classify them with a different cutoff (say 0.7).
4. Construct a confusion matrix using this new cutoff.

### **Task 5: ROC and Precision-Recall Curve Analysis**

1. Calculate the area under the precision-recall curve.
2. Compare the AUC from the ROC and the precision-recall curve. What can you conclude from the differences or similarities in the AUC values?
3. Change the logistic regression model (e.g., add or remove predictors) and observe how it affects the ROC curve and precision-recall curve.

In [5]:
## Read data
LHON = read.table("https://raw.githubusercontent.com/luuloi/GWAS_Introduction_2023/main/data/LHON.txt", header=TRUE, stringsAsFactors=TRUE)

In [6]:
## Check first few rows
head(LHON)

,IID,GENO,PHENO
,<fct>,<fct>,<fct>
1,ID1,TT,CONTROL
2,ID2,CT,CONTROL
3,ID3,TT,CASE
4,ID4,CT,CONTROL
5,ID5,TT,CONTROL
6,ID6,TT,CONTROL


In [ ]:
# First create a genotype variable with an additive coding based on the counts of the number of T alleles
LHON$genoadd <- with(LHON, 0 + 1*(GENO=="CT") + 2*(GENO=="TT"))
head(LHON)

# 1.1 Check the last few rows of the LHON dataframe

In [ ]:
tail()

# 1.2 Check the unique values in the GENO and PHENO columns

In [ ]:
unique(LHON$)
unique(LHON$)

# 1.3 Calculate the mean, median, and mode

In [ ]:
mean(LHON$genoadd)
median(LHON$genoadd)

get_mode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}
mode_genoadd <- get_mode(LHON$)

# 2.1 Pie chart for the PHENO column

In [ ]:
pie(table(LHON$), main="Distribution of PHENO", col=c("lightblue", "lightcoral"))

# 2.2 Histogram for numeric column

In [ ]:
hist(LHON$, main="Distribution of genoadd", col="lightgreen")

# 2.3 Scatter plot between two numeric columns (assuming we have another numeric column 'numeric_column_2')

In [ ]:
# For this demonstration, let's use genoadd vs. genoadd (just to showcase the command)
library(ggplot2)
ggplot(LHON, aes(x=genoadd, y=genoadd, color=)) + geom_point() + theme_minimal()

# 3.1 Contingency table

In [ ]:
# Assuming we use GENO and newpheno for the contingency table
cont_table_new <- table(LHON$, LHON$)

# 3.2 Chi-square test

In [ ]:
chi_test_new <- chisq.test(cont_table_new)
print(chi_test_new)

# 4.1 Adding genoadd to the model

In [ ]:
logistmod4 <- glm(newpheno ~ genoadd + , family=binomial(link="logit"), data=LHON)

# 4.2 Interpret the summary

In [ ]:
summary()

# 4.3 Predict using the model

In [ ]:
LHON$pred_new <- predict(, type="response")

# Classification with 0.7 as the cutoff
LHON$predicted_pheno <- ifelse(LHON$pred_new > 0.7, "CASE", "CONTROL")

# 4.4 Confusion matrix with new cutoff

In [ ]:
confusionMatrix(table(LHON$predicted_pheno, LHON$))

# 5.1 Area under the precision-recall curve

In [ ]:
# Install precrec package
pacman::p_load("precrec")

# loading library
library(precrec)

ROCnPR_new <- evalmod(scores = LHON$pred_new, labels = LHON$)
auc_new <- auc(ROCnPR_new)
auc_new[which(auc_new$type == "PRC"),]

# 5.2 Compare AUC from ROC and precision-recall curve

In [ ]:
print(auc_new)

# 5.3 Change logistic regression model (here, we remove GENO)

In [ ]:
logistmod5 <- glm(newpheno ~ genoadd, family=binomial(link="logit"), data=LHON)
LHON$pred5 <- predict(logistmod5, type="response")
ROCnPR5 <- evalmod(scores = LHON$pred5, labels = LHON$)
autoplot(ROCnPR5, "ROC")
autoplot(ROCnPR5, "PRC")